In [1]:
##aim of this code is to develop a simple LSTM neural network that can identify hate speech

##relevant imports
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import LSTM, Dense

Using TensorFlow backend.


In [2]:
from keras.preprocessing.sequence import pad_sequences

In [3]:
from keras.callbacks import LearningRateScheduler

In [4]:
from keras.optimizers import Adam

In [5]:
from IPython.display import clear_output

In [ ]:
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("labeled_data.csv")

In [ ]:
#create a list of sentences where each word is represented by a spacy word vector
x =[]
c=0
m= len(df["tweet"])
for t in df["tweet"]:
    x.append(np.asarray([t.vector for t in nlp(t)]))
    c+=1
    print("%.4f" % (c/m * 100), "%")
    clear_output(wait=True)

In [5]:
wordcount = 40

In [ ]:
#pads and trunactes the sentences the sentence vectors
# 40 is somewhat aribtrary
for i, s in enumerate(x):
    x[i] = pad_sequences([x[i]],maxlen=wordcount, padding = "pre",dtype='float32', truncating = "post")[0]

In [ ]:
x = np.asarray(x)

In [ ]:
#The output data is based on the "hate_speech" column of the dataset.
y= list(df["hate_speech"])

In [ ]:
#In the original dataset, however, this "hate_speech" column is not binary:
set(df["hate_speech"])
#As explained in the source of the dataset, the value of this column is based on how many CrowdFlower users deemed it to be hate speech

In [ ]:
#it will be assumed for the purposes of this project that texts that at least one CF user identified as hate speech are indeed hate speech
#as this is a binary cross entropy task, two categories are  created
for i, v in enumerate(y):
    #not hate speech
    if v == 0:
        y[i] = np.asarray([1,0])
    #hate speech
    elif v>0:
        y[i] = np.asarray([0,1])

In [ ]:
#converts y from a list into a numpy array
y = np.asarray(y)

In [ ]:
#splits the data into traning and testing data
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state = 42)

In [ ]:
#saves the vectorised sentences
np.save("HateSpeechTestingData.npy", X_test)

In [6]:
#for loading the training data for a new session
X_train = np.load("HateSpeechTrainingData.npy")

In [7]:
y_train = np.load("HatespeechYtrain.npy")

In [16]:
#decreases the learning rate exponentially after 10 epochs
def scheduler(epoch):
    if epoch < 10:
        print(0.00016529890126548707)
        return 0.00016529890126548707
    else:
        lrnrt=  float(0.001 * tf.math.exp(0.1 * (10 - (epoch+ 29))))
        print(lrnrt)
        return lrnrt

In [17]:
learnr = LearningRateScheduler(scheduler)

In [18]:
#the model
model = Sequential()
model.add(LSTM(256, input_shape = (wordcount, 300)))
model.add(Dense(300, activation = 'relu'))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))

In [19]:
optimizer = Adam(lr=0.00016529890126548707)
model.compile(loss = 'binary_crossentropy', optimizer=optimizer)

In [21]:
model.fit(X_train, y_train, batch_size = 50, epochs = 50, callbacks =[learnr])

Epoch 1/50
0.00016529890126548707
16604/16604 [==============================] - 88s 5ms/step - loss: 0.0044
Epoch 2/50
0.00016529890126548707
16604/16604 [==============================] - 89s 5ms/step - loss: 0.0051
Epoch 3/50
0.00016529890126548707
16604/16604 [==============================] - 94s 6ms/step - loss: 0.0043
Epoch 4/50
0.00016529890126548707
16604/16604 [==============================] - 95s 6ms/step - loss: 0.0039
Epoch 5/50
0.00016529890126548707
16604/16604 [==============================] - 95s 6ms/step - loss: 0.0033
Epoch 6/50
0.00016529890126548707
16604/16604 [==============================] - 94s 6ms/step - loss: 0.0033
Epoch 7/50
0.00016529890126548707
16604/16604 [==============================] - 93s 6ms/step - loss: 0.0034
Epoch 8/50
0.00016529890126548707
16604/16604 [==============================] - 93s 6ms/step - loss: 0.0033
Epoch 9/50
0.00016529890126548707
16604/16604 [==============================] - 93s 6ms/step - loss: 0.0037
Epoch 10/50
0.00016

In [ ]:
#ended on epoch 29
#lr = 0.00016529890126548707

In [24]:
X_test = np.load("HateSpeechTestingData.npy")

In [26]:
#generates predictions for the model
preds = model.predict(X_test)

In [27]:
from sklearn.metrics import classification_report

In [28]:
#converts the prediction of the model into a string
def choice(arr):
    if arr[0] > arr[1]:
        return "not hate"
    else:
        return "hate"

In [29]:
predictions = [choice(x) for x in preds]
actualresults = [choice(x) for x in y_test]

In [25]:
y_test= np.load("HatespeechYtest.npy")

In [30]:
#prints out a classification report for the model
print(classification_report(predictions, actualresults))

              precision    recall  f1-score   support

        hate       0.41      0.49      0.44      1375
    not hate       0.89      0.86      0.87      6804

    accuracy                           0.79      8179
   macro avg       0.65      0.67      0.66      8179
weighted avg       0.81      0.79      0.80      8179



In [ ]:
#saves the model weights
model.save_weights("HateSpeechLSTMweights06032.ckpt")

In [20]:
#loads the model weights
model.load_weights("HateSpeechLSTMweights0603.ckpt")